In [0]:

%run "/Workspace/my_databricks_learning/databricks_learning/Usecases/4_logistics_usecase_generic/generic_func_nb/generic_functions_nb"

In [0]:
%python
from pyspark.sql.functions import *
def staff_data_standardization(df):
    return (
        df.withColumn("role",lower(col("role")))
        .withColumn("hub_location",initcap(col("hub_location")))
        .withColumn("age",word_to_num_udf(col("age")).cast("int"))
        .withColumn("shipment_id",word_to_num_udf(col("shipment_id"))
        .cast("int"))
        .withColumnRenamed("first_name","staff_first_name")
        .withColumnRenamed("last_name","staff_last_name")
        .withColumnRenamed("hub_location","origin_hub_city"))

In [0]:
def logistics_shipment_standarisation(df):
    return (
        df.withColumn("domain",lit("Logistics"))
        .withColumn("is_expedited",lit(False))
        .withColumn("ingestion_timestamp",lit(current_timestamp()))
        .withColumn("vehicle_type",upper(col("vehicle_type")))
        .withColumn("shipment_date",to_date("shipment_date","yy-MM-dd"))
        .withColumn("shipment_cost",round(col("shipment_cost"),2))
        .withColumn("shipment_weight_kg",col("shipment_weight_kg").cast("double"))
        .withColumn("is_expedited",col("is_expedited").cast("boolean")))

In [0]:
def staff_enriched_data(df):
    return (
        df.withColumn("load_dt",lit(current_timestamp()))
        .withColumn("full_name",concat(col("staff_first_name"),lit(" "),col("staff_last_name")))
        .select('shipment_id','full_name', 'age', 'role', 'origin_hub_city', 'vehicle_type', 'load_dt','data_source'))

In [0]:
def logistics_shipment_enriched_data(df):
    return 
    (
        df.withColumn("route_segment",concat(col("source_city"),lit("-"),col("destination_city")))
        .withColumn("vehicle_identifier",concat(col("vehicle_type"),lit("_"),col("shipment_id")))
        .withColumn("shipment_year",year(col("shipment_date")))
        .withColumn("shipment_month",month(col("shipment_date")))
        .withColumn("is_weekend",when((dayofweek(col("shipment_date")) == 1) | (dayofweek(col("shipment_date")) == 7 ),True).otherwise(False))
        .withColumn("is_expedited",when((col("shipment_status") == 'IN_TRANSIT') | (col("shipment_status") == 'DELIVERED'), True).otherwise(False))
        .withColumn("cost_per_kg", try_divide(col("shipment_cost"), col("shipment_weight_kg")))
        .withColumn("days_since_shipment",datediff(current_date(),col("shipment_date")))
        .withColumn("tax_amount",col("shipment_cost")*0.18)
        .withColumn("order_prefix", substring(col("order_id"), 1, 3))
        .withColumn("order_sequence", substring(col("order_id"), 4, length(col("order_id"))))
        .withColumn("spilt_date",split(col("shipment_date"),"-"))
        .withColumn("ship_year",col("spilt_date")[0])
        .withColumn("ship_month",col("spilt_date")[1])
        .withColumn("ship_day",col("spilt_date")[2]).drop("spilt_date")
        .withColumn("route_lane",concat(col("source_city"),lit("->"),col("destination_city")))
        )